In [1]:
# use if autocompletion is not working
%config Completer.use_jedi = False

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
from tqdm import tqdm

import numpy as np

from copy import deepcopy
from typing import List
from collections import OrderedDict

In [4]:
from modern_slavery_registry import get_root_path

In [5]:
PROJECT_PATH = get_root_path()
DATA_PATH = os.path.join(PROJECT_PATH, "data")
SHEETS_PATH = os.path.join(PROJECT_PATH, "data", "sheets")

In [6]:
os.listdir(SHEETS_PATH)

['subset_data_v4.xlsx',
 '.ipynb_checkpoints',
 'Continents_to_CountryNames.csv',
 'fuzzy_matches.xlsx',
 'combined_modern_slavery_statements.xlsx',
 'modern_slavery_dataset.csv',
 'modern_slavery_dataset_nitin.xlsx',
 'subset_data.xlsx',
 'CountryCodes_to_CountryNames.csv',
 'subset_data_v3.xlsx',
 'subset_data_v2.xlsx',
 'ext_shawn_creds.csv']

## Mapping continents to country codes

In [7]:
df = pd.read_excel(os.path.join(SHEETS_PATH, "subset_data_v4.xlsx"))
df.drop([col for col in df.columns if col.endswith("_None")], inplace=True, axis=1)

In [8]:
df.head()

,final_statement,Company,FS_industry,FS_hq,FS_uk_modern_slavery_act,FS_california_transparency_in_supply_chains_act,FS_australia_modern_slavery_act,FS_period_covered,year,HQ_KZ,...,SUBSECTOR_Aircraft/Airline,SUBSECTOR_Education companies,SUBSECTOR_Entertainment,SUBSECTOR_Gambling,SUBSECTOR_Packaging,SUBSECTOR_Clothing & textile,"SUBSECTOR_Oil, gas & coal",SUBSECTOR_Catering & food services,SUBSECTOR_Industrial gases,statement
0,1Spatial Modern Slavery Act Policy Statement H...,1Spatial Plc,Software,United Kingdom,1.0,0.0,0.0,2017.0,NaN,0,...,0,0,0,0,0,0,0,0,0,home solution government boundary law enforcem...
1,Slavery and Human Trafficking Statement – Shaz...,1Stop Halal Limited,Food Products,United Kingdom,1.0,0.0,0.0,2018.0,2019.0,0,...,0,0,0,0,0,0,0,0,0,shazans shazan food continue monitor covid 19 ...
2,7/28/2019 Modern Slavery Statement 2018 - 1st ...,1st Step Solutions Limited,Professional Services,United Kingdom,1.0,0.0,0.0,2018.0,2019.0,0,...,0,0,0,0,0,0,0,0,0,28 2019 2018 statement make pursuant sec 54 20...
3,fh Modern Slavery Act 2015: slavery and human ...,2 Agriculture Limited,Food Products,United Kingdom,1.0,0.0,0.0,2018.0,2019.0,0,...,0,0,0,0,0,0,0,0,0,fh 2015 introduction uk act require business s...
4,1 Modern Slavery Act 2015: slavery and human t...,2 Agriculture Limited,Food Products,United Kingdom,1.0,0.0,0.0,2017.0,2018.0,0,...,0,0,0,0,0,0,0,0,0,2015 introduction uk act require business stat...


In [9]:
mappingCountryCodes2CountryNames = pd.read_csv(os.path.join(SHEETS_PATH, "CountryCodes_to_CountryNames.csv"))
mappingCountryCodes2CountryNames.drop([col for col in mappingCountryCodes2CountryNames.columns if col.startswith("Unnamed")], inplace=True, axis=1)
mappingCountryCodes2CountryNames.set_index("Country", inplace=True)
mappingCountryCodes2CountryNames.dropna(inplace=True)
mappingCountryCodes2CountryNames.head()

,Iso2c
Country,
Afghanistan,AF
Albania,AL
Algeria,DZ
American Samoa,AS
Andorra,AD


In [10]:
mappingContinent2CountryNames = pd.read_csv(os.path.join(SHEETS_PATH, "Continents_to_CountryNames.csv"))
mappingContinent2CountryNames.drop([col for col in mappingContinent2CountryNames.columns if col.startswith("Column")], axis=1, inplace=True)
mappingContinent2CountryNames.set_index("Country", inplace=True)
mappingContinent2CountryNames.dropna(inplace=True)
mappingContinent2CountryNames.head()

,Continent
Country,
Algeria,Africa
Angola,Africa
Benin,Africa
Botswana,Africa
Burkina,Africa


In [11]:
mappingCountryCodes2Continents = pd.merge(mappingCountryCodes2CountryNames, mappingContinent2CountryNames, on="Country", how="inner")
mappingCountryCodes2Continents.reset_index(drop=True)
mappingCountryCodes2Continents.set_index("Iso2c", inplace=True)
mappingCountryCodes2Continents.head()

,Continent
Iso2c,
AF,Asia
AL,Europe
DZ,Africa
AD,Europe
AO,Africa


In [12]:
dfCountryCodes = pd.DataFrame(data=[col[3:] for col in df.columns if col.startswith("HQ_")], columns=["Iso2c"])
dfCountryCodes.set_index("Iso2c", inplace=True)
print(f"Found {len(dfCountryCodes)} countries")
dfCountryCodes.head()

Found 55 countries


""
Iso2c
KZ
RU
CY
US
GG


In [13]:
dfCountryCodes2Continents = pd.merge(dfCountryCodes, mappingCountryCodes2Continents, on="Iso2c", how="left")
print(f"Missing continents: {dfCountryCodes2Continents['Continent'].isna().sum()}")
dfCountryCodes2Continents.head()

Missing continents: 9


,Continent
Iso2c,
KZ,Asia
RU,NaN
CY,Europe
US,North America
GG,NaN


In [14]:
dfCountryCodes2Continents[dfCountryCodes2Continents["Continent"].isna()].index

Index(['RU', 'GG', 'KR', 'GL', 'TW', 'BM', 'HK', 'IO', 'JE'], dtype='object', name='Iso2c')

In [15]:
missingdfCountryCodes2Continents = pd.DataFrame.from_dict(
    {"IO": "Asia",
     "GG":"Europe",
     "RU": "Asia", 
     "JE": "Europe", 
     "HK": "Asia",
     "GL": "North America", 
     "TW": "Asia", 
     "KR": "Asia", 
     "BM": "North America"},columns=["Continent"], 
    orient="index")
missingdfCountryCodes2Continents.index.rename("Iso2c", inplace=True)
completeCountryCodes2Continents =  pd.concat([dfCountryCodes2Continents[~dfCountryCodes2Continents["Continent"].isna()], missingdfCountryCodes2Continents])
completeCountryCodes2Continents.sort_index(inplace=True)
print(
    f"Number of Countries: {len(completeCountryCodes2Continents.index.dropna())}, "
    f"Number of Continents: {len(completeCountryCodes2Continents['Continent'].dropna())}")
completeCountryCodes2Continents.head()

Number of Countries: 55, Number of Continents: 55


,Continent
Iso2c,
AE,Asia
AF,Asia
AT,Europe
AU,Oceania
BE,Europe


In [16]:
onehotCompleteCountryCodes2Continents = pd.get_dummies(completeCountryCodes2Continents)
onehotCompleteCountryCodes2Continents.head()

,Continent_Africa,Continent_Asia,Continent_Europe,Continent_North America,Continent_Oceania,Continent_South America
Iso2c,,,,,,
AE,0,1,0,0,0,0
AF,0,1,0,0,0,0
AT,0,0,1,0,0,0
AU,0,0,0,0,1,0
BE,0,0,1,0,0,0


In [17]:
df[[f"HQ_{c}" for c in completeCountryCodes2Continents.index]].head()

,HQ_AE,HQ_AF,HQ_AT,HQ_AU,HQ_BE,HQ_BH,HQ_BM,HQ_BR,HQ_CA,HQ_CH,...,HQ_SA,HQ_SE,HQ_SG,HQ_TH,HQ_TR,HQ_TW,HQ_UG,HQ_US,HQ_VN,HQ_ZA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
final_df = pd.concat([
    df,
    pd.DataFrame(
        data=np.dot(
            df[[f"HQ_{c}" for c in completeCountryCodes2Continents.index]].values, 
            onehotCompleteCountryCodes2Continents.values), 
        columns=[f"HQ_{col.replace(' ', '_')}" for col in onehotCompleteCountryCodes2Continents.columns])], 
    axis=1)
final_df.fillna("#NA", inplace=True)
final_df.head()

,final_statement,Company,FS_industry,FS_hq,FS_uk_modern_slavery_act,FS_california_transparency_in_supply_chains_act,FS_australia_modern_slavery_act,FS_period_covered,year,HQ_KZ,...,"SUBSECTOR_Oil, gas & coal",SUBSECTOR_Catering & food services,SUBSECTOR_Industrial gases,statement,HQ_Continent_Africa,HQ_Continent_Asia,HQ_Continent_Europe,HQ_Continent_North_America,HQ_Continent_Oceania,HQ_Continent_South_America
0,1Spatial Modern Slavery Act Policy Statement H...,1Spatial Plc,Software,United Kingdom,1.0,0.0,0.0,2017.0,#NA,0,...,0,0,0,home solution government boundary law enforcem...,0,0,1,0,0,0
1,Slavery and Human Trafficking Statement – Shaz...,1Stop Halal Limited,Food Products,United Kingdom,1.0,0.0,0.0,2018.0,2019.0,0,...,0,0,0,shazans shazan food continue monitor covid 19 ...,0,0,1,0,0,0
2,7/28/2019 Modern Slavery Statement 2018 - 1st ...,1st Step Solutions Limited,Professional Services,United Kingdom,1.0,0.0,0.0,2018.0,2019.0,0,...,0,0,0,28 2019 2018 statement make pursuant sec 54 20...,0,0,1,0,0,0
3,fh Modern Slavery Act 2015: slavery and human ...,2 Agriculture Limited,Food Products,United Kingdom,1.0,0.0,0.0,2018.0,2019.0,0,...,0,0,0,fh 2015 introduction uk act require business s...,0,0,1,0,0,0
4,1 Modern Slavery Act 2015: slavery and human t...,2 Agriculture Limited,Food Products,United Kingdom,1.0,0.0,0.0,2017.0,2018.0,0,...,0,0,0,2015 introduction uk act require business stat...,0,0,1,0,0,0


In [19]:
final_df.to_excel(os.path.join(SHEETS_PATH, "subset_data_v5.xlsx"), index=False)